In [1]:
import pandas as pd
import logging
from scrapy.crawler import CrawlerProcess
import os 
import scrapy
import re
import json
from pandas_dq import dq_report

In [2]:
# Loading the table sorted according to the elast rainy destinations
city_df=pd.read_csv('./city_rain_avg.csv', index_col=False)

In [3]:
# Drop indexes columns, Should have been done before when saving
city_df=city_df.drop(city_df.columns[0:2], axis=1)

In [4]:
## I WILL NEED TO REMOVE THE INDEX COLUMNS, EASY BUT I FORGOT HOW TO
city_df.head(10)

,name,place_id,lat,lon,city,pop
0,Bormes-les-Mimosas,101877567,43.150697,6.341928,Bormes-les-Mimosas,0.0320
1,Aix-en-Provence,102741999,43.529842,5.447474,Aix-en-Provence,0.0590
2,Saintes-Maries-de-la-Mer,102529062,43.451592,4.427720,Saintes-Maries-de-la-Mer,0.1000
3,Avignon,102193665,43.949249,4.805901,Avignon,0.1005
4,Nîmes,102307633,43.837425,4.360069,Nîmes,0.1280
5,Cassis,102437647,43.214036,5.539632,Cassis,0.1360
6,Aigues-Mortes,102716851,43.566152,4.191540,Aigues-Mortes,0.1390
7,Collioure,103975931,42.525050,3.083155,Collioure,0.1410
8,Marseille,387540349,43.296174,5.369953,Marseille,0.1455
9,Uzès,106427400,44.012128,4.419672,Uzès,0.1480


In [5]:
# Slice the dataframe to the top 5 least rainy cities
city_df_trim=city_df[:5]

In [6]:
city_df_trim

,name,place_id,lat,lon,city,pop
0,Bormes-les-Mimosas,101877567,43.150697,6.341928,Bormes-les-Mimosas,0.0320
1,Aix-en-Provence,102741999,43.529842,5.447474,Aix-en-Provence,0.0590
2,Saintes-Maries-de-la-Mer,102529062,43.451592,4.427720,Saintes-Maries-de-la-Mer,0.1000
3,Avignon,102193665,43.949249,4.805901,Avignon,0.1005
4,Nîmes,102307633,43.837425,4.360069,Nîmes,0.1280


In [7]:
# capture the 5 least rainy cities
city5=city_df_trim['name']
city5

0          Bormes-les-Mimosas
1             Aix-en-Provence
2    Saintes-Maries-de-la-Mer
3                     Avignon
4                       Nîmes
Name: name, dtype: object

In [8]:
## Testing the new loop, div starts at 3 and goes down the list by a step value of 2, so 5th hotel should be 11
for i in range(3, 13, 2):
    print(i)

3
5
7
9
11


In [9]:
#test
city_df_trim.iloc[1]['name']

'Aix-en-Provence'

In [10]:
for i in range(len(city_df_trim)):
    print(city_df_trim.iloc[i]['name'])

Bormes-les-Mimosas
Aix-en-Provence
Saintes-Maries-de-la-Mer
Avignon
Nîmes


In [11]:
# Booking URL filtered by the top ratings, CITY to replace with city in city5
# 'https://www.booking.com/searchresults.en-gb.html?aid=304142&label=gen173nr-1FCAQoggJCDnNlYXJjaF9hdmlnbm9uSAlYBGhNiAEBmAEJuAEXyAEM2AEB6AEB-AEDiAIBqAIDuAK89suzBsACAdICJDJhNjlmZWQ4LTMwNjEtNDY3NC04MTMyLWVkYjIzZjZhYmIwOdgCBeACAQ&ss={}&order=class'.format(city)


## Final Cell with GPS

In [15]:
# ## Cannot do a loop for each 5 cities because kernel needs to be restarted each time we launch spiders. There are onlky 5 cities to scrap hotels from, so it is easier to write the start URLs by hand
# #Python script for testing, no need to restart

from scrapy.crawler import CrawlerProcess
import scrapy
import os
import logging

class BookingSpider(scrapy.Spider):
    name = 'nested_spider'
    download_delay = 2.5
    start_urls = [
            'https://www.booking.com/searchresults.en-gb.html?ss={}'.format(city_df_trim.iloc[0]['name']),
            'https://www.booking.com/searchresults.en-gb.html?ss={}'.format(city_df_trim.iloc[1]['name']),
            'https://www.booking.com/searchresults.en-gb.html?ss={}'.format(city_df_trim.iloc[2]['name']),
            'https://www.booking.com/searchresults.en-gb.html?ss={}'.format(city_df_trim.iloc[3]['name']),
            'https://www.booking.com/searchresults.en-gb.html?ss={}'.format(city_df_trim.iloc[4]['name']),
    ]

    def parse(self, response):
        for i in range(3, 13, 2):
            item = {
                'city': response.xpath('/html/body/div[4]/div/div[2]/div/div[2]/div[3]/div[2]/div[2]/div[3]/div[{}]/div[1]/div[2]/div/div/div[1]/div/div[2]/div/a/span/span[1]/text()'.format(i)).get(),
                'name': response.xpath('/html/body/div[4]/div/div[2]/div/div[2]/div[3]/div[2]/div[2]/div[3]/div[{}]/div[1]/div[2]/div/div/div[1]/div/div[1]/div/h3/a/div[1]/text()'.format(i)).get(),
                'user_reviews': response.xpath('/html/body/div[4]/div/div[2]/div/div[2]/div[3]/div[2]/div[2]/div[3]/div[{}]/div[1]/div[2]/div/div/div[2]/div/div[1]/a/span/div/div[1]/text()'.format(i)).get(),
                'url': response.xpath('/html/body/div[4]/div/div[2]/div/div[2]/div[3]/div[2]/div[2]/div[3]/div[{}]/div[1]/div[2]/div/div/div[1]/div/div[1]/div/h3/a'.format(i)).css('::attr(href)').get(),
                'description': response.xpath('/html/body/div[4]/div/div[2]/div/div[2]/div[3]/div[2]/div[2]/div[3]/div[{}]/div[1]/div[2]/div/div/div[1]/div/div[3]/text()'.format(i)).get(),
            }

            url = item['url']
            if url:
                yield response.follow(url, callback=self.parse_nested_page, meta={'item': item})

    def parse_nested_page(self, response):
        item = response.meta['item']
        item['hotel_gps'] = response.xpath('/html/body/div[4]/div/div[4]/div[1]/div[1]/div[1]/div[1]/div[3]/div[4]/p/a').css('::attr(data-atlas-latlng)').get()
        yield item

# Name of the file where the results will be saved
filename = "Booking_nested_top5_final.json"

# Ensure 'src/' directory exists
os.makedirs('src', exist_ok=True)

# If file already exists, delete it before crawling (because Scrapy will concatenate the last and new results otherwise)
if filename in os.listdir('src/'):
    os.remove('src/' + filename)

# Declare a new CrawlerProcess with some settings
process = CrawlerProcess(settings={
    'USER_AGENT': 'Chrome/126.0.0.0',
    'LOG_LEVEL': logging.INFO,
    "FEEDS": {
        'src/' + filename: {"format": "json"},
    }
})

# Start the crawling using the spider you defined above
process.crawl(BookingSpider)
process.start()


2024-06-25 23:08:18 [scrapy.utils.log] INFO: Scrapy 2.11.2 started (bot: scrapybot)
2024-06-25 23:08:18 [scrapy.utils.log] INFO: Versions: lxml 5.2.2.0, libxml2 2.11.7, cssselect 1.2.0, parsel 1.9.1, w3lib 2.2.1, Twisted 24.3.0, Python 3.12.4 (tags/v3.12.4:8e8a4ba, Jun  6 2024, 19:30:16) [MSC v.1940 64 bit (AMD64)], pyOpenSSL 24.1.0 (OpenSSL 3.2.2 4 Jun 2024), cryptography 42.0.8, Platform Windows-11-10.0.22631-SP0
2024-06-25 23:08:18 [scrapy.addons] INFO: Enabled addons:
[]
2024-06-25 23:08:18 [py.warnings] WARNING: C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\scrapy\utils\request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compati

In [43]:
temp_json = open('C:/Users/romai/Documents/Data_Science_Courses/Fullstack/Jedha_Fullstack_Data_Science/Project/Kayak/src/25-Booking_nested_top5_final.json')
hotels = json.load(temp_json)

In [12]:
hotels

[{'city': 'Bormes-les-Mimosas',
  'name': 'La petite Provence',
  'user_reviews': '8.8',
  'url': 'https://www.booking.com/hotel/fr/lovely-place-for-2.en-gb.html?aid=304142&label=gen173nr-1FCAQoggJCGXNlYXJjaF9ib3JtZXMtbGVzLW1pbW9zYXNICVgEaE2IAQGYAQm4AQrIAQXYAQHoAQH4AQOIAgGoAgO4Avzd7LMGwAIB0gIkMzEwMGQzMmItNWIwMS00ZDA4LWJkM2QtNjU3YzNhYTI1YzA42AIF4AIB&ucfs=1&arphpl=1&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=1&hapos=1&sr_order=popularity&srpvid=2ae9933e89bc00f9&srepoch=1719348990&from=searchresults',
  'description': 'Set in Bormes-les-Mimosas, 29 km from Chateau de Grimaud and 29 km from Le Pont des Fées, La petite Provence offers city views and free WiFi. ',
  'hotel_gps': None},
 {'city': 'Bormes-les-Mimosas',
  'name': 'Hôtel Paradis',
  'user_reviews': '8.4',
  'url': 'https://www.booking.com/hotel/fr/paradis.en-gb.html?aid=304142&label=gen173nr-1FCAQoggJCGXNlYXJjaF9ib3JtZXMtbGVzLW1pbW9zYXNICVgEaE2IAQGYAQm4AQrIAQXYAQHoAQH4AQOIAgGoAgO4Avzd7LMGwAIB0gIk

In [44]:
hotel_df=pd.DataFrame(hotels)

In [14]:
hotel_df

,city,name,user_reviews,url,description,hotel_gps
0,Bormes-les-Mimosas,La petite Provence,8.8,https://www.booking.com/hotel/fr/lovely-place-for-2.en-gb.html?aid=304142&label=gen173nr-1FCAQoggJCGXNlYXJjaF9ib3JtZXMtbGVzLW1pbW9zYXNICVgEaE2IAQGYAQm4AQrIAQXYAQHoAQH4AQOIAgGoAgO4Avzd7LMGwAIB0gIkMzEwMGQzMmItNWIwMS00ZDA4LWJkM2QtNjU3YzNhYTI1YzA42AIF4AIB&ucfs=1&arphpl=1&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=1&hapos=1&sr_order=popularity&srpvid=2ae9933e89bc00f9&srepoch=1719348990&from=searchresults,"Set in Bormes-les-Mimosas, 29 km from Chateau de Grimaud and 29 km from Le Pont des Fées, La petite Provence offers city views and free WiFi.",None
1,Bormes-les-Mimosas,Hôtel Paradis,8.4,https://www.booking.com/hotel/fr/paradis.en-gb.html?aid=304142&label=gen173nr-1FCAQoggJCGXNlYXJjaF9ib3JtZXMtbGVzLW1pbW9zYXNICVgEaE2IAQGYAQm4AQrIAQXYAQHoAQH4AQOIAgGoAgO4Avzd7LMGwAIB0gIkMzEwMGQzMmItNWIwMS00ZDA4LWJkM2QtNjU3YzNhYTI1YzA42AIF4AIB&ucfs=1&arphpl=1&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=5&hapos=5&sr_order=popularity&srpvid=2ae9933e89bc00f9&srepoch=1719348990&from_sustainable_property_sr=1&from=searchresults,Hôtel Paradis is set in a building built in 1920 and has a luxuriant garden and offers a panoramic view over the sea and the old village of Bormes.,None
2,Bormes-les-Mimosas,300 m plages commerces,8.8,https://www.booking.com/hotel/fr/studio-design-bormes-les-mimosas.en-gb.html?aid=304142&label=gen173nr-1FCAQoggJCGXNlYXJjaF9ib3JtZXMtbGVzLW1pbW9zYXNICVgEaE2IAQGYAQm4AQrIAQXYAQHoAQH4AQOIAgGoAgO4Avzd7LMGwAIB0gIkMzEwMGQzMmItNWIwMS00ZDA4LWJkM2QtNjU3YzNhYTI1YzA42AIF4AIB&ucfs=1&arphpl=1&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=4&hapos=4&sr_order=popularity&srpvid=2ae9933e89bc00f9&srepoch=1719348990&from=searchresults,"Set in Bormes-les-Mimosas, just 400 metres from Faviere, 300 m plages commerces offers beachfront accommodation with a private beach area, a bar and free WiFi.",None
3,Bormes-les-Mimosas,Hotel Restaurant Bellevue,7.5,https://www.booking.com/hotel/fr/restaurant-bellevue.en-gb.html?aid=304142&label=gen173nr-1FCAQoggJCGXNlYXJjaF9ib3JtZXMtbGVzLW1pbW9zYXNICVgEaE2IAQGYAQm4AQrIAQXYAQHoAQH4AQOIAgGoAgO4Avzd7LMGwAIB0gIkMzEwMGQzMmItNWIwMS00ZDA4LWJkM2QtNjU3YzNhYTI1YzA42AIF4AIB&ucfs=1&arphpl=1&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=3&hapos=3&sr_order=popularity&srpvid=2ae9933e89bc00f9&srepoch=1719348990&from=searchresults,"Located in downtown Bormes-les-Mimosas, Hotel Restaurant Bellevue features a restaurant serving fresh and local products, a bar and a sitting area in front of the property.",None
4,Bormes-les-Mimosas,Bormes - Le Loft,8.8,https://www.booking.com/hotel/fr/bormes-le-loft.en-gb.html?aid=304142&label=gen173nr-1FCAQoggJCGXNlYXJjaF9ib3JtZXMtbGVzLW1pbW9zYXNICVgEaE2IAQGYAQm4AQrIAQXYAQHoAQH4AQOIAgGoAgO4Avzd7LMGwAIB0gIkMzEwMGQzMmItNWIwMS00ZDA4LWJkM2QtNjU3YzNhYTI1YzA42AIF4AIB&ucfs=1&arphpl=1&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=2&hapos=2&sr_order=popularity&srpvid=2ae9933e89bc00f9&srepoch=1719348990&from_sustainable_property_sr=1&from=searchresults,"Offering city views, Bormes - Le Loft is an accommodation set in Bormes-les-Mimosas, 33 km from Le Pont des Fées and 39 km from Toulon Train Station.",None
5,Aix-en-Provence,B&B Villa Roumanille,9.3,https://www.booking.com/hotel/fr/villa-roumanille.en-gb.html?aid=304142&label=gen173nr-1FCAQoggJCFnNlYXJjaF9haXgtZW4tcHJvdmVuY2VICVgEaE2IAQGYAQm4AQrIAQXYAQHoAQH4AQOIAgGoAgO4Av_d7LMGwAIB0gIkNjA5NDhjZDUtODU3Yy00ZmI4LTg3NGYtYjM4OTAzZDU4YWVh2AIF4AIB&ucfs=1&arphpl=1&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=1&hapos=1&sr_order=popularity&srpvid=98f9933fcabd0312&srepoch=1719348992&from_sustainable_property_sr=1&from=searchresults,"Located just 1 km from the centre of Aix-en-Provence, Villa Roumanille is a typical Provençal house, which offers a garden and terrace. Cassis and its cliffs are a 30-minute

In [15]:
dq_report(hotel_df)

This is a summary report. Change verbose to 1 to see more details on each DQ issue.
    All variables classified into correct types.


C:\Users\romai\AppData\Local\Temp\ipykernel_17560\3147084410.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dq_report(hotel_df)


,DQ Issue
The Good News,"There are no duplicate rows in this dataset, There are no duplicate columns in this datatset, There are no ID columns in the dataset., There are no zero-variance or low information columns in the dataset., There are no date-time vars in this dataset, There are no columns with infinite values in this dataset , There are no high cardinality columns in this dataset, There are no highly correlated columns in the dataset., There is no target given. Hence no target leakage columns detected in the dataset"
The Bad News,There are 3 columns with mixed data types in the dataset. Convert them to a single type or split them into multiple columns.


,Data Type,Missing Values%,Unique Values%,Minimum Value,Maximum Value,DQ Issue
city,object,0.0,32,,,No issue
name,object,0.0,100,,,No issue
user_reviews,object,4.0,56,,,"1 missing values. Impute them with mean, median, mode, or a constant value such as 123., Mixed dtypes: has 2 different data types: object,"
url,object,0.0,100,,,No issue
description,object,4.0,96,,,"1 missing values. Impute them with mean, median, mode, or a constant value such as 123., Mixed dtypes: has 2 different data types: object,"
hotel_gps,object,40.0,60,,,"10 missing values. Impute them with mean, median, mode, or a constant value such as 123., Mixed dtypes: has 2 different data types: object,"


There are missing GPS coordinates for two cities, I am dropping them because I will still have 15 hotels without rain to choose from !

In [45]:
## Filter for hotel with missing gps coordinates
condition=(hotel_df['hotel_gps'].isna() == False)


In [46]:
hotel_df = hotel_df[condition]
hotel_df.reset_index(inplace=True, drop=True)
hotel_df

,city,name,user_reviews,url,description,hotel_gps
0,Aix-en-Provence,B&B Villa Roumanille,9.3,https://www.booking.com/hotel/fr/villa-roumanille.en-gb.html?aid=304142&label=gen173nr-1FCAQoggJCFnNlYXJjaF9haXgtZW4tcHJvdmVuY2VICVgEaE2IAQGYAQm4AQrIAQXYAQHoAQH4AQOIAgGoAgO4Av_d7LMGwAIB0gIkNjA5NDhjZDUtODU3Yy00ZmI4LTg3NGYtYjM4OTAzZDU4YWVh2AIF4AIB&ucfs=1&arphpl=1&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=1&hapos=1&sr_order=popularity&srpvid=98f9933fcabd0312&srepoch=1719348992&from_sustainable_property_sr=1&from=searchresults,"Located just 1 km from the centre of Aix-en-Provence, Villa Roumanille is a typical Provençal house, which offers a garden and terrace. Cassis and its cliffs are a 30-minute drive away.","43.51783611,5.44283509"
1,"Aix-en-Provence Historic Centre, Aix-en-Provence",Aquabella Hôtel & Spa,8.6,https://www.booking.com/hotel/fr/hotel-aquabella.en-gb.html?aid=304142&label=gen173nr-1FCAQoggJCFnNlYXJjaF9haXgtZW4tcHJvdmVuY2VICVgEaE2IAQGYAQm4AQrIAQXYAQHoAQH4AQOIAgGoAgO4Av_d7LMGwAIB0gIkNjA5NDhjZDUtODU3Yy00ZmI4LTg3NGYtYjM4OTAzZDU4YWVh2AIF4AIB&ucfs=1&arphpl=1&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=5&hapos=5&sr_order=popularity&srpvid=98f9933fcabd0312&srepoch=1719348992&from_sustainable_property_sr=1&from=searchresults,"Surrounded by a beautiful garden, Aquabella offers Provencal-style rooms and guests have free access to the spa. It is located in central Aix-en-Provence, a short walk from the old town.","43.53126360,5.44507004"
2,"Aix-en-Provence Historic Centre, Aix-en-Provence",Hôtel des Augustins,8.1,https://www.booking.com/hotel/fr/les-augustins.en-gb.html?aid=304142&label=gen173nr-1FCAQoggJCFnNlYXJjaF9haXgtZW4tcHJvdmVuY2VICVgEaE2IAQGYAQm4AQrIAQXYAQHoAQH4AQOIAgGoAgO4Av_d7LMGwAIB0gIkNjA5NDhjZDUtODU3Yy00ZmI4LTg3NGYtYjM4OTAzZDU4YWVh2AIF4AIB&ucfs=1&arphpl=1&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=4&hapos=4&sr_order=popularity&srpvid=98f9933fcabd0312&srepoch=1719348992&from=searchresults,"Located just of the famous Cours Mirabeau in the heart of the old town of Aix-en-Provence, Hotel Les Augustins is set within the walls of a 12th-century convent.","43.52688664,5.44727147"
3,Aix-en-Provence,Hôtel Le Mozart,8.4,https://www.booking.com/hotel/fr/le-mozart.en-gb.html?aid=304142&label=gen173nr-1FCAQoggJCFnNlYXJjaF9haXgtZW4tcHJvdmVuY2VICVgEaE2IAQGYAQm4AQrIAQXYAQHoAQH4AQOIAgGoAgO4Av_d7LMGwAIB0gIkNjA5NDhjZDUtODU3Yy00ZmI4LTg3NGYtYjM4OTAzZDU4YWVh2AIF4AIB&ucfs=1&arphpl=1&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=3&hapos=3&sr_order=popularity&srpvid=98f9933fcabd0312&srepoch=1719348992&from=searchresults,"Located 500 metres from the Granet Museum, Hôtel Le Mozart offers air-conditioned rooms with free WiFi. It is easily accessed from Motorway A8, a 3-minute drive away.","43.52191762,5.45787156"
4,Aix-en-Provence,Séjours & Affaires Aix-en-Provence Mirabeau,7.8,https://www.booking.com/hotel/fr/residence-mirabeau.en-gb.html?aid=304142&label=gen173nr-1FCAQoggJCFnNlYXJjaF9haXgtZW4tcHJvdmVuY2VICVgEaE2IAQGYAQm4AQrIAQXYAQHoAQH4AQOIAgGoAgO4Av_d7LMGwAIB0gIkNjA5NDhjZDUtODU3Yy00ZmI4LTg3NGYtYjM4OTAzZDU4YWVh2AIF4AIB&ucfs=1&arphpl=1&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=2&hapos=2&sr_order=popularity&srpvid=98f9933fcabd0312&srepoch=1719348992&from_sustainable_property_sr=1&from=searchresults,This Séjours & Affaires residence is located in Aix-en-Provence near the Cours Mirabeau. It offers furnished studios and free Wi-Fi internet access.,"43.52524078,5.44126600"
5,"Avignon City Centre, Avignon","Au cœur des Papes, dans bâtisse de charme studios & appartements en cœur de ville",8.7,https://www.booking.com/hotel/fr/au-coeur-des-papes.en-gb.html?aid=304142&label=gen173nr-1FCAQoggJCDnNlYXJjaF9hdmlnbm9uSAlYBGhNiAEBmAEJuAEKyAEF2AEB6AEB-AEDiAIBqAIDuAKE3uyzBsACAdICJDdkNDE2YWZkLTA1MWItNGMyMy05NjY1LTIyZDgxN2RlZGNjZdgCBeACAQ&ucfs=1&arphpl=1&group_adults=2&req_adults=2&no_rooms=1&group_children=